In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import download
from string import digits
import re
import random
import numpy as np
import itertools

### 1. Load the data into a Python data frame.

In [3]:
fin = pd.read_table("/Users/Chewei/Desktop/UK/ML/smsspamcollection/SMSSpamCollection.csv", header=None)
fin["filterm"] = None
fin.columns = ["label","original","filterm"] # rename columns to make it clear
print(fin.head())

  label                                           original filterm
0   ham  Go until jurong point, crazy.. Available only ...    None
1   ham                      Ok lar... Joking wif u oni...    None
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...    None
3   ham  U dun say so early hor... U c already then say...    None
4   ham  Nah I don't think he goes to usf, he lives aro...    None


### 2. Pre-process the SMS messages: Remove all punctuation and numbers from the SMS messages, and change all messages to lower case. (Please provide the Python code that achieves this!)

In [4]:
k = 0
for sentence in fin["original"]:
    filtered_sentence =[]
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]','',sentence) #removes all punctuation from sentence
    sentence = sentence.translate({ord(k): None for k in digits}) # removes numbers from sentence
    words = word_tokenize(sentence)
    fin.loc[k,"filterm"]=words
    k = k+1
    
for i, w in enumerate(fin.filterm):
    w= ' '.join(word for word in fin.filterm[i])
    fin.filterm[i]=w
    
print(fin.head())

  label                                           original  \
0   ham  Go until jurong point, crazy.. Available only ...   
1   ham                      Ok lar... Joking wif u oni...   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3   ham  U dun say so early hor... U c already then say...   
4   ham  Nah I don't think he goes to usf, he lives aro...   

                                             filterm  
0  go until jurong point crazy available only in ...  
1                            ok lar joking wif u oni  
2  free entry in a wkly comp to win fa cup final ...  
3        u dun say so early hor u c already then say  
4  nah i dont think he goes to usf he lives aroun...  


### 3. Shuffle the messages and split them into a training set (2,500 messages), a validation set (1,000 messages) and a test set (all remaining messages).

In [5]:
def train_validate_test_split(df, train_percent=(2500/5571), validate_percent=(1000/5571), seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

In [36]:
train, validation, test = train_validate_test_split(fin,seed=19890724)

In [37]:
print(train.head(2))
print(validation.head(2))
print(test.head(2))

     label                                           original  \
4962   ham  A bit of Ur smile is my hppnss, a drop of Ur t...   
3681   ham  I cant pick the phone right now. Pls send a me...   

                                                filterm  
4962  a bit of ur smile is my hppnss a drop of ur te...  
3681  i cant pick the phone right now pls send a mes...  
     label                                           original  \
2845   ham  Today iZ Yellow rose day. If u love my frndshi...   
2611   ham     As usual..iam fine, happy &amp; doing well..:)   

                                                filterm  
2845  today iz yellow rose day if u love my frndship...  
2611              as usualiam fine happy amp doing well  
     label                                           original  \
2273   ham  Haha awesome, I've been to 4u a couple times. ...   
1701   ham                    Please ask mummy to call father   

                                                filterm  
2273  

### 5. Explain the code: What is the purpose of each function? What do ’train’ and ‘train2’ do, and what is the difference between them? Where in the code is Bayes’ Theorem being applied?

In [30]:
class NaiveBayesForSpam :
    def train ( self , hamMessages , spamMessages) : 
        self.words = set ('_'.join (hamMessages + spamMessages).split()) #Create a set consisting of 
        #all the words that occur in the mails
        self.priors = np.zeros (2) # probability of being spam / ham without further knowledge
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len ( spamMessages ) )
        # the above one is P(ham)
        self.priors [1] = 1.0 - self.priors[0]  # P(spam)
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len ( hamMessages )
            # if word of "words" is in hamMessages, then add one to the numerator, after looping
            # this number will be divided by the total word of hamMessages
            # = P(word|ham)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) /len ( spamMessages )
            # if word of "words" is in spamMessages, then add one to the numerator, after looping
            # this number will be divided by the total word of spamMessages
            # = P(word|spam)
            self.likelihoods.append ([min (prob1 , 0.95) , min (prob2 ,0.95) ])
            # when some words do not appear in ham or spam, the probability will be 1. That is the probability 
            #of occurance is 0. To avoid a 0 divisor we the Laplace tranformation is applied.
        self.likelihoods = np.array (self.likelihoods).T
            # just transpose the dataframe for further use
        
    
    def train2(self, hamMessages, spamMessages):
        self.words = set(''.join(list(hamMessages) + list(spamMessages)).split()) #Create a set consisting of 
        #all the words that occur in the mails
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages)) #P(ham) 
        self.priors[1] = 1.0 - self.priors[0] #P(spam)
        self.likelihoods = []
        spamkeywords = [ ]
        for i, w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages) #P(W|ham)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages)  #P(W|spam)
            if prob1 * 20 < prob2:
                # if probability of a word being spam is 20 times higher than being ham (that is, 20 times more
                #more likely to appear in spam than in ham)
                #then this word will be taken into the dataframe of spamkeywords
                self.likelihoods.append([min(prob1 , 0.95) , min(prob2 , 0.95)]) #Laplace transformation
                spamkeywords.append(w) 
        self.words = spamkeywords
                # update the words using only highly (filtered) spam words
        self.likelihoods = np.array(self.likelihoods).T
                # just transpose the dataframe for further use
        
    def predict(self, message):
        posteriors = np.copy(self.priors)
            # copy P(ham), P(spam)
        for i, w in enumerate(self.words):
            # using words in train one, using spam keywords in train two
            if w in message.lower(): 
                # convert to lower−case
                posteriors *= self.likelihoods[:,i] 
                # using bayes' theorem and conditional independence here
                # P(viagara,paypal,now | spam) is propotional to 
                #              P(viagara|spam)*P(paypal|spam)*P(now|spam)*P(spam)
                # the first three probability is in self.likehoods, and the last one is in posteriors
            else:
                posteriors *= np.ones(2) - self.likelihoods[:,i] 
                # if the word is not in the message
                # then the posteriors multiply P(not the word|spam ) or P(not the word|ham)
            posteriors = posteriors / np.linalg.norm(posteriors, ord = 1) 
                #normalise using max(sum(abs(x), axis=0))
        if posteriors[0] > 0.5:
            # set the cutoff value  0.5
            # if probability of being ham is greater than 0.5, them we say this is a ham mail
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]] 

    def score(self, messages, labels):
        confusion = np.zeros(4).reshape(2, 2) 
        num=0
        for m, l in zip(messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham': 
                confusion[0 ,0] += 1
            # if we predict the message is ham and it is actually ham, the +1 in the true positive 
            elif self.predict(m)[0] == 'ham' and l == 'spam': 
                confusion[0 ,1] += 1
            # if we predict the message is ham and it is actually spam, the +1 in the false negative 
            elif self.predict(m)[0] == 'spam' and l == 'ham': 
                confusion[1 ,0] += 1
            # if we predict the message is spam and it is actually ham, the +1 in the false positive
            elif self.predict(m)[0] == 'spam' and l == 'spam': 
                confusion[1 ,1] += 1
            # if we predict the message is ham and it is actually ham, the +1 in the true negative 
        return (confusion[0,0] + confusion[1,1]) / float(confusion.sum()), confusion
            # add up the diagonal figures and divide by total numbers of cases

### 6. Use your training set to train the classifiers ‘train’ and ‘train2’. Note that the interfaces of our classifiers require you to pass the ham and spam messages separately.

In [31]:
# Since the train function will take spam and ham separately, here we are going to sparate these two kinds of messages 
train_spam = train[train.label=="spam"]["filterm"]
train_ham = train[train.label=="ham"]["filterm"]
# Creating the list of strings to be used for training
train_spam  = list(itertools.chain(train_spam))
train_ham = list(itertools.chain(train_ham))

##### Following is the accuracy rate based on train set. The accurancy on validation set in next paragraph. 

In [32]:
SpamFilter = NaiveBayesForSpam() # initiate the function
SpamFilter.train(train_ham,train_spam)  
score, cm = SpamFilter.score(train.filterm, train.label)

In [33]:
print('accuracy:', score)
print()
print('confusion matrix:')
print(cm)

accuracy: 0.972

confusion matrix:
[[ 2139.    39.]
 [   31.   291.]]


In [38]:
#train2
SpamFilter = NaiveBayesForSpam()
SpamFilter.train2(train_ham,train_spam)
score2, cm2 = SpamFilter.score(train.filterm, train.label)

In [39]:
print('accuracy:', score2)
print()
print('confusion matrix:')
print(cm2)

accuracy: 0.9724

confusion matrix:
[[ 2163.    60.]
 [    9.   268.]]


### 7. Using the validation set, explore how each of the two classifiers performs out of sample.

In [23]:
score_vd, cm_vd = SpamFilter.score(validation.filterm, validation.label)

In [24]:
print('accuracy:', score_vd)
print()
print('confusion matrix:')
print(cm_vd)

accuracy: 0.956

confusion matrix:
[[ 843.   34.]
 [  10.  113.]]


In [25]:
SpamFilter.train2(train_ham,train_spam)

In [26]:
score_vd2, cm_vd2 = SpamFilter.score(validation.filterm, validation.label)

In [27]:
print('accuracy:', score_vd2)
print()
print('confusion matrix:')
print(cm_vd2)

accuracy: 0.959

confusion matrix:
[[ 852.   40.]
 [   1.  107.]]


The accuracy for train() and train2() for validation set is 0.956 and 0.959 respectively. The accuracy for train2() is higher. train() may have resulted in overfitting, hence resulting in lower accuracy for validation set.

### 8. Why is the ‘train2’ classifier faster? Why does it yield a better accuracy both on the training and the validation set?

By including a marker (print(i)) for the number of iterations in the predict() function, we can see that there is a significant reduction in the number of iterations when we use train2() instead of train().

In [ ]:
def predict(self, message):
        posteriors = np.copy(self.priors)
        for i, w in enumerate(self.words):
            if w in message.lower():
                posteriors *= self.likelihoods[:, i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors / np.linalg.norm(posteriors)
            print(i) # prints the number of iterations
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]

Using the message "Money Viagra" as an input to predict() function, there are 5501 iterations when using train() and 172 iterations when using train2(). Fewer iterations are achieved because train2() stores only the spam key words for later comparison with new records as compared to storing both ham and spam words encountered in the training set. Therefore, train2 classifier is fast.

train2() is likely to yield a better accuracy for both the training and the validation set. This is because train2() assigns a word to spamkeywords list when prob1*20 < prob2.  The value of 20 in prob1 *20 < prob2 determines the accuracy of the spamkeywords list. In turn, the accuracy of the list of spam key words will then impact the accuracy of new records being accurately classified as spam or ham. 

However, it should be noted that train2() uses less predictors(i.e. spamkeywords) than train(), the accuracy may not always be higher.


### 9. How many false positives (ham messages classified as spam messages) did you get in your validation set? How would you change the code to reduce false positives at the expense of possibly having more false negatives (spam messages classified as ham messages)?

With 'train' classifier, the false positive is 34 and the false positive of train two is 49. The reason is that train two only tests those highly-spam keyword and thus has higher accuarcy. 

To reduce the false positives at the expense of having more false negatives, we can adjust the threshold of a message being classified as spam from 0.50 to 0.70. This adjustment requires a message to have more spam words before being classified as a spam message. The accuracy after adjusting the threshold is reported below.

In [42]:
class NaiveBayesForSpam_edited :
    def train ( self , hamMessages , spamMessages) : 
        self.words = set ('_'.join (hamMessages + spamMessages).split())
        self.priors = np.zeros (2) # probability of being spam / ham without further knowledge
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len ( spamMessages ) )
        # the above one is P(ham)
        self.priors [1] = 1.0 - self.priors[0]  # P(spam)
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len ( hamMessages )
            # if word of "words" is in hamMessages, then add one to the numerator, after looping
            # this number will be divided by the total word of hamMessages
            # = P(word|ham)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) /len ( spamMessages )
            # if word of "words" is in spamMessages, then add one to the numerator, after looping
            # this number will be divided by the total word of spamMessages
            # = P(word|spam)
            self.likelihoods.append ([min (prob1 , 0.95) , min (prob2 ,0.95) ])
            # when some words do not appear in ham or spam, the probability will be 1. The above line
            # avoid this situation
        self.likelihoods = np.array (self.likelihoods).T
            # just transpose the dataframe for further use
        
    
    def train2(self, hamMessages, spamMessages):
        self.words = set(''.join(list(hamMessages) + list(spamMessages)).split())
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages))
        self.priors[1] = 1.0 - self.priors[0] 
        self.likelihoods = []
        spamkeywords = [ ]
        for i, w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages) 
            if prob1 * 20 < prob2:
                self.likelihoods.append([min(prob1 , 0.95) , min(prob2 , 0.95)])
                spamkeywords.append(w) 
                # if probability of a word being spam is 20 times higher tham being ham
                # then this word will be taken into the dataframe of spamkeywords
        self.words = spamkeywords
                # update the words unsing only highly spam words
        self.likelihoods = np.array(self.likelihoods).T
                # just transpose the dataframe for further use
        
    def predict(self, message):
        posteriors = np.copy(self.priors)
            # copy P(ham), P(spam)
        for i, w in enumerate(self.words):
            # using words in train one, using spam keywords in train two
            if w in message.lower(): 
                # convert to lower−case
                posteriors *= self.likelihoods[:,i] 
                # using bayes' theorem here
                # P(viagara,paypal,now | spam) is propotional to 
                #              P(viagara|spam)*P(paypal|spam)*P(now|spam)*P(spam)
                # the first three probability is in self.likehoods, and the last one is in posteriors
            else:
                posteriors *= np.ones(2) - self.likelihoods[:,i] 
                # if the word is not in the message
                # then the posteriors multipy P(not the word|spam ) or P(not the word|ham)
            posteriors = posteriors / np.linalg.norm(posteriors, ord = 1) 
                #normalise using max(sum(abs(x), axis=0))
        if posteriors[0] > 0.7:
            # set the cutoff value  0.5
            # if probability of being ham is greater than 0.5, them we say this is a ham mail
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]] 

    def score(self, messages, labels):
        confusion = np.zeros(4).reshape(2, 2) 
        num=0
        for m, l in zip(messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham': 
                confusion[0 ,0] += 1
            # if the we predict the message is ham and it is actually ham, the +1 in the true positive 
            elif self.predict(m)[0] == 'ham' and l == 'spam': 
                confusion[0 ,1] += 1
            # if the we predict the message is ham and it is actually spam, the +1 in the false negative 
            elif self.predict(m)[0] == 'spam' and l == 'ham': 
                confusion[1 ,0] += 1
            # if the we predict the message is spam and it is actually ham, the +1 in the false positive
            elif self.predict(m)[0] == 'spam' and l == 'spam': 
                confusion[1 ,1] += 1
            # if the we predict the message is ham and it is actually ham, the +1 in the true negative 
        return (confusion[0,0] + confusion[1,1]) / float(confusion.sum()), confusion
            # add up the diagonal figures and divide by total numbers of cases

In [47]:
SpamFilter_edited = NaiveBayesForSpam_edited ()

In [49]:
SpamFilter_edited.train(train_ham,train_spam)
score_p1, cm_p1 = SpamFilter_edited.score(validation.filterm, validation.label)
print('accuracy:', score_p1)
print()
print('confusion matrix:')
print(cm_p1)

accuracy: 0.965

confusion matrix:
[[ 841.   23.]
 [  12.  124.]]


The original false positive is 34, and the improved version is 23. But the false negative increases from 10 to 12. 

In [48]:
SpamFilter_edited.train2(train_ham,train_spam)
score_p2, cm_p2 = SpamFilter.score(validation.filterm, validation.label)
print('accuracy:', score_p2)
print()
print('confusion matrix:')
print(cm_p2)

accuracy: 0.96

confusion matrix:
[[ 852.   39.]
 [   1.  108.]]


There is a slight improvement for train2(). The original false positive is 40, and the improved version is 39. False negative reamins as 1. 

### 10. Run the ‘train2’ classifier on the test set and report its performance using a confusion matrix.

In [40]:
SpamFilter.train2(train_ham,train_spam)
score_test, cm_test = SpamFilter.score(test.filterm, test.label)
print('accuracy:', score_test)
print()
print('confusion matrix:')
print(cm_test)

accuracy: 0.97249034749

confusion matrix:
[[ 1794.    51.]
 [    6.   221.]]


In [ ]:
The accuracy on test set is 97.249%